디스크립터는 다른 언어에서도 생소한 개념이어서 마땅한 비교대상이 없다.

디스크립터는 파이썬의 객체지향 수준을 한 단계 더 끌어올려주는 혁신적인 기능으로 이 기능을 잘 활용하면 보다 견고하고 재사용성이 높은 추상화를 할 수 있다. 디스크립터의 기능을 제대로 활용하는 예는 라이브러리나 프레임워크에서 많이 발견할 수 있다.

* 디스크립터가 무엇인지 어떻게 동작하는지 어떻게 효율적으로 구현하는지 이해한다.
* 두 가지 유형의 디스크립터 - 데이터 디스크립터와 비데이터 디스크립터의 개념적 차이와 세부구현의 차이를 분석한다.
* 디스크립터를 화용한 코드 재사용 방법
* 디스크립터의 좋은 사용 예를 살펴보고 자체 라이브러리의 API에 어떻게 활용할 수 있는지 살펴본다.

## 디스크립터 개요

먼저 디스크립터에 대한 배경지식을 쌓은 다음 보다 깔끔하고 파이썬스러운 구현을 한 예제를 살펴본다.

### 디스크립터 메커니즘

디스크립터의 동작방식은 그리 복잡하지 않지만, 세부 구현 시의 주의사항이 많다는 점에 유의해야 한다.

디스크립터를 구현하려면 최소 두 개의 클래스가 필요하다. 클라이언트 클래스는 디스크립터 구현의 기능을 활용할 도메인 모델로서 솔루션을 위해 생성한 일반적인 추상화 객체이다. 디스크립터 클래스는 디스크립터 로직의 구현체이다. 

디스크립터는 단지 디스크립터 프로토콜을 구현한 클래스의 인스턴스이다.이 클래스는 다음 매직 메서드 중에 최소 한 개 이상을 포함해야 한다.

* \__get__
* \__set__
* \__delete__
* \__set__name__

다음과 같은 네이밍 컨벤션을 사용한다.

* ClientClass - 디스크립터 구현체의 기능을 활용할 도메인 추상화 객체. 디스크립터의 클라이언트이다. 클래스 속성(인스턴스 속성과 달리 여러 객체가 값을 공유함)으로 디스크립터를 갖는다. 디스크립터는 DescriptorClass의 인스턴스이다.

* DescriptorClass - 디스크립터 클래스. 이 클래스는 앞으로 언급할 디스크립터 프로토콜을 따르는 매직 메서드를 구현해야만 한다.

* client - ClientClass의 인스턴스

* descriptor - DescriptorClass의 인스턴스

명심해야 할 중요한 사실을 이 프로토콜이 동작하려면 디스크립터 객체가 클래스 속성으로 정의되어야 한다는 것이다. 이 객체를 인스턴스 속성으로 생성하면 동작하지 않으므로 init 메서드가 아니라 클래스 본문에 있어야 한다.

디스크립터 프로토콜의 일부만 구현해도 된다는 것에 유의하자. 

디스크립터 클래스와 디스크립터의 로직을 사용하는 크랠스, 즉 디스크립터 객체를 멤버로 갖는 클래스, 이렇게 두 개의 클래스가 있다. ClientClass의 인스턴스에서 descriptor 속성을 호출하면 디스크립터 프로토콜이 사용된다. 이 모든 동작은 러타임 중에 어떻게 작동할까?

다음은 일반적인 클래스의 속성 또는 프로퍼티에 접근하면 예상한 것과 같은 결과를 얻을 수 있다.

In [1]:
class Attribute:
    value = 42

class Client:
    attribute = Attribute()

Client.attribute

In [2]:
Client.attribute.value

42

그러나 디스크립터의 경우 약간 다르게 동작한다. 클래스 속성을 객체로 선언하면 디스크립터로 인식되고, 클라이언트에서 해당 속성을 호출하면 객체 자체를 반환하는 것이 아니라 \__get__ 매직 메서드의 결과를 반환한다.

호출당시의 문맥 정보를 로깅하고 클라이언트 인스턴스를 그대로 반환하는 간단한 예제를 살펴보자.

In [13]:
import logging
import time
import functools
logger = logging.getLogger(__name__)

class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        logger.info("Call: %s.__get__(%r, %r)" %(self.__class__.__name__, instance, owner))
        print("Call: %s.__get__(%r, %r)" %(self.__class__.__name__, instance, owner))

        return instance

class ClientClass:
        descriptor = DescriptorClass()

이제 ClientClass 인스턴스의 descriptor 속성에 접근해보면 DescriptorClass 인스턴스를 반환하지 않고 대신에 \__get__() 메서드의 반환 값을 사용한다는 것을 알 수 있다.

In [14]:
client = ClientClass()
client.descriptor

Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x06CCF930>, <class '__main__.ClientClass'>)


In [15]:
client.descriptor is client

Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x06CCF930>, <class '__main__.ClientClass'>)


True

\__get__ 메서드에 있던 로그가 어떻게 찍혔는지 살펴보자. 이 예제에서는 클라이언트 자체를 그대로 반환했으므로 마지막 비교 문장은 True가 된다. \__get__ 메서드의 파라미턴는 이후 섹션에서 각 메서드별로 자세히 살펴본다. 

간단한 설명용 예제에서 출발하여 더 복잡한 추상화와 더 나은 데코레이터를 만들어 볼 것이다. 디스크립터를 사용하면 완전히 새롭게 프로그램의 제어 흐름을 변경할 수 있다. 이 도구를 사용해 \__get__ 메서드 뒤쪽으로 모든 종류의 논리를 추상화할 수 있으며 클라이언트에게 내용을 숨긴 채로 모든 유형의 변환을 투명하게 실행할 수 있다. 

### 디스크립터 프로토콜의 메서드 탐색

디스크립터의 내부 동작 원리에 대해 알아보자. 디스크립터는 단지 객체이기 때문에 이러한 메서드들은 self를 첫 번째 파라미터로 사용한다. self는 디스크립터 객체 자신을 의미한다. 

디스크립터 프로토콜의 각 메서드에 사용되는 파라미터와 사용 방법에 대해서 알아보자.

### \__get__(self, instance, owner)

두 번째 파라미터 instance는 디스크립터를 호출한 객체를 의미한다. 앞선 예제에서는 client 객체를 의미한다.

owner 파라미터는 해당 객체의 클래스를 의미한다. 이어질 예제에서는 ClientClass로 앞엣 살펴보았던 클래스이다.

앞에서 \__get__ 메서드 시그니처에 있는 instance 파라미터는 디스크립터가 행동을 취하려는 객체이고 owner는 인스턴스의 클래스이다. owner = instance.\__class__와 같은 형태로 instance에서 클래스를 직접 구할 수 있음에도 굳이 owner를 시그니처에 사용하는 이유는 client 인스턴스가 아니라 ClientClass에서 descriptor를 호출하는 특별한 경우로서 instance의 값은 None이기 때문에 클래스를 구할 수 없고, 따라서 굳이 파라미터를 추가하여 owner를 받은 것이다.

간단한 코드를 사용하여 디스크립터가 클래스에서 호출될 때와 인스턴스에서 호출될 때의 차이를 알아보자. \__get__ 메서드는 각각의 경우에 대해 두 개의 개별적인 작업을 수행한다.

In [37]:
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return f"{self.__class__.__name__}.{owner.__name__}"
        return f"value for {instance}"

class ClientClass:
    descriptor = DescriptorClass()

In [38]:
ClientClass.descriptor

'DescriptorClass.ClientClass'

In [39]:
ClientClass().descriptor

'value for <__main__.ClientClass object at 0x06C3EA50>'

일반적으로 정말 owner 파라미터를 사용하려는 경우가 아니라면, 인스턴스가 None일 때는 단순히 디스크립터 자체를 반환한다.

### \__set__(self, instance, value)

이 메서드는 디스크립터에 값을 할당하려고 할 때 호춛된다. 다음과 같이 \__set__() 메서드를 구현한 디스크립터에 대해서만 활성화된다. 다음 예제의 경우 instance 파라미터는 client이고 value는 "value"라는 문자열이다.

In [40]:
client.descriptor = "value"

In [41]:
client.descriptor

'value'

client.descriptorsms \__set__() 메서드를 구현하지 않았다. 따라서 descriptor 자체를 덮어쓴다.

다음은 이 메서드를 활용하여 속성의 유효성을 검사하는 객체를 어떻게 만들 수 있는지 보여준다. 유효성 검사 함수는 자유롭게 생성할 수 있으며 객체에 값을 할당하기 전에 실행된다. 

In [44]:
class Validation:
    def __init__(self, validation_function, error_msg: str):
        self.validation_function = validation_function
        self.error_msg = error_msg
    
    def __call__(self, value):
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")

class Field:
    def __init__(self, *validations):
        self._name = None
        self.validations = validations
    
    def __set_name__(self, owner, name):
        self._name = name
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]
    
    def validate(self, value):
        for validation in self.validations:
            validation(value)
    
    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self._name] = value

class ClientClass:
    descriptor = Field(
        Validation(lambda x : isinstance(x, (int, float)), "는 숫자가 아님"),
        Validation(lambda x : x >= 0, "는 0보다 작음"),
    )

위 코드를 실행 하면 다음과 같은 결과가 나온다.

In [45]:
client = ClientClass()
client.descriptor = 42

In [46]:
client.descriptor

42

In [47]:
client.descriptor = -42

ValueError: -42 는 0보다 작음

In [48]:
client.descriptor = 'invalid value'

ValueError: 'invalid value' 는 숫자가 아님

여기서 기억할 점은 프로퍼티 자리에 놓일 수 있는 것은 디스크립터로 추상화할 수 있으며 여러 번 재사용할 수 있다는 것이다. 이 예에서는 \__set__() 메서드가 @property.setter가 하던 일을 대신한다.

### \__delete__(self, instance)

이 메서드는 다음과 같은 형태로 호출된다. self는 descriptor 속성을 나타낸고 instance는 client를 나타낸다.

In [50]:
del client.descriptor

다음 예제에서는 이 메서드를 사용하여 관리자 권한이 없는 객체에서 속성을 제거하지 못하도록 하는 디스크립터를 만들 것이다. 특히 디스크립터가 클라이언트 객체의 소성 값을 어떻게 다루고 있는지 유심히 살펴보자.

In [53]:
class ProtectedAttribute:
    def __init__(self, requires_role=None) -> None:
        self.permission_required = requires_role
        self._name = None
    
    def __set_name__(self, owner, name):
        self._name = name
    
    def __set__(self, user, value):
        if value is None:
            raise ValueError(f"{self._name}를 None으로 설정할 수 없음")
        user.__dict__[self._name] = value
    
    def __delete__(self, user):
        if self.permission_required in user.permissions:
            user.__dict__[self._name] = None
        else:
            raise ValueError(
                f"{user!s} 사용자는 {self.permission_required} 권한이 없음"
            )
class User:
    """admin  권한을 가진 사용자만 이메일 주소를 삭제할 수 있음."""
    
    email = ProtectedAttribute(requires_role="admin")
    
    def __init__(self, username: str, email: str, permission_list: list = None) -> None:
        self.username = username
        self.email = email
        self.permissions = permission_list or []
    
    def __str__(self):
        return self.username

객체가 어떻게 동작하는지 살펴보기 전에 몇 가지 미스크립터의 기준을 확인하는 것이 중요하다. User클래스는 username과 email을 필수로 받는다. email 속성을 지워버리면 불완전한 객체가 되고 User 클래스에서 정의한 인터페이스와 맞지 않는 유효하지 않은 상태가 된다. 문제를 예방하기 위해선 이 같은 세부 사항을 확인하는 것이 중요하다. User를 사용하고자 하는 객체는 email 속성이 있는 것으로 기대하고 있다. 

이런 이유 때문에 email의 "삭제"는 단순히 None으로 설정하는 것으로 한다. admin 권한을 가진 사용자만 email 주소를 제가할 수 있다고 가정하면 다음과 같이 동작하는 것을 볼 수 있다.

In [54]:
admin = User('root', "root@d.com", ["admin"])
user = User("user", "user1@d.com", ["email", "helpdesk"])

In [55]:
admin.email

'root@d.com'

In [56]:
del admin.email

In [57]:
admin.email is None

True

In [58]:
user.email

'user1@d.com'

In [59]:
user.email = None

ValueError: email를 None으로 설정할 수 없음

In [60]:
del user.email

ValueError: user 사용자는 admin 권한이 없음

일반적으로 \__delete__ 메서드는 앞의 두 메서드에 비해 자주 사용되지는 않지만 이해의 완결성을 위해 살펴보는 것이 좋다. 

### \__set_name__(self, owner, name)

일반적으로 클래스에 디스크립터 객체를 만들 때는 디스크립터가 처리하려는 속성의 이름을 알아야 한다.

속성의 이름은 \__dict__에서 \__get__과 \__set__ 메서드로 읽고 쓸 때 사용된다.

파이썬 3.6 이전에는 디스크립터가 이 이름을 자동으로 설정하지 못했기 때문에 보통은 객체 초기화 시 명시적으로 이름을 전다랬다. 이렇게 해도 잘 동작하지만 새로운 속성에 대한 디스크립터를 추가할 때마다 이름을 복사해야 하는 불편함이 있었다.

다음은 \__set_name__이 없을 때의 전형적인 디스크립터 코드이다.

In [73]:
class DescriptorWithName:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, value):
        if instance is None:
            return self
        logger.info("%r에서 %r 속성 가져오기" %(instance, self.name))
        print("%r에서 %r 속성 가져오기" %(instance, self.name))
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

class ClientClass:
    descriptor = DescriptorWithName("descriptor")

In [74]:
client = ClientClass()

In [75]:
client.descriptor = "value"

In [76]:
client.descriptor

<__main__.ClientClass object at 0x06C0B830>에서 'descriptor' 속성 가져오기


'value'

속성의 이름을 두 번 쓰지 않으려면, 다음과 같이 몇 가지 트릭을 사용해야 한다. 클래스 데코레이터를 사용하거나 심지어는 메타클래스를 사용해야 한다. 

파이썬 3.6에서 새로운 메서드 \__set_name__이 추가되었는데 이 메서드는 파라미터로 디스크립터를 소유한 클래스와 디스크립터의 이름을 받는다. 디스크립터에 이 메서드를 추가하여 필요한 이름을 지정하면 된다.

일반적으로 하위 호환을 위해 \__init__ 메서드에 기본 값을 지정하고 \__set_name__을 함께 사용하는 것이 좋다.

이 방법을 사용하면 다음과 같이 다시 작성할 수 있다.

In [154]:
class DescriptorWithName:
    def __init__(self, name=None):
        self.name = name
    
    def __set_name__(self, owner, name):
        self.name = name
        
class ClientClass:
    descriptor = DescriptorWithName()

In [155]:
client = ClientClass()

In [156]:
client.descriptor = "value"

In [157]:
client.descriptor

'value'

이제 번거롭게 변수 이름을 복사하여 파라미터에 전달하지 않아도 된다.

## 디스크립터의 유형

방금 살펴본 메서드를 사용하는 디스크립터의 작동방식에 따라 디스크립터를 구분할 수 있다. 이 구분을 이해하는 것은 디스크립터를 효과적으로 사용하는데 중요한 역할을 하고, 런타임 시 경고 또는 일반적인 오류를 피하는데 도움이 된다.

디스크립터가 \__set__이나 \__delete__ 메서드를 구현했다면 데이터 디스크립터라고 부른다. 그렇지 않고 \__get__만을 구현한 디스크립터를 비데이터 디스크립터라고 부른다. \__set_name__은 이 분류에 전형 영향을 미치지 않는다.

객체의 속성을 결정할 때 데이터 디스크립터가 객체의 사전보다 우선적으로 적용되지만 비데이터 디스크립터는 그렇지 않다. 즉 비데이터 디스크립터는 객체의 사전에 디스크립터와 동일한 이름의 키가 있으면 객체의 사전 값이 적용되고 디스크립터는 절대 호출되지 않을 것이다. 반대로, 데이터 디스크립터에서는 디스크립터와 동일한 이름을 갖는 키가 사저네 존재하더라도 디스크립터 자체가 항상 먼저 호출되기 때문에 객체의 키 값은 결코 사용되지 않을 것이다.

### 비데이터 디스크립터

먼저 다음 예제를 통해 \__get__ 메서드만을 구현한 디스크립터를 살펴보자.

In [158]:
class NoneDataDescriptor:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 42

class ClientClass:
    descriptor = NoneDataDescriptor()

평소처럼 descriptor를 호출하면 \__get__ 메서드의 결과를 얻을 수 있다.

In [159]:
client = ClientClass()
client.descriptor

42

descriptor 속성을 다른 값으로 바꾸면 이전의 값을 읽고 대신에 새로 설정한 값을 얻는다.

In [160]:
client.descriptor = 43

In [161]:
client.descriptor

43

이제 descriptor를 지우고 다시 물으면 어떤 값을 얻게 될까?

In [162]:
del client.descriptor

In [163]:
client.descriptor

42

In [164]:
del client.descriptor

AttributeError: descriptor

방금 일어난 일을 살펴보면, 처음 client 객체를 만들었을 때 descriptor 속성은 인스턴스가 아니라 클래스 안에 있다. 따라서 client 객체의 사전을 조회하면 그 값은 비어 있다.

In [92]:
vars(client)

{}

여기서 .descriptor 속성을 조회하면 client.\__dict__에서 "discriptor"라는 이름의 키를 찾지 못하고 마침내 클래스에서 디스크립터를 찾게 된다. 이것이 \__get__ 메서드의 결과가 반환되는 이유이다.

그러나 .descriptor 속성에 다른 값을 설정하면 인스턴스의 사전이 변경되므로 이제 client.\__dict__는 비어 있지 않다.

In [93]:
client.descriptor = 99
vars(client)

{'descriptor': 99}

따라서 이제 .desciptor 속성을 조회하면 객체의 \__dict__ 사전에 descriptor 키를 찾을 수 있으므로 클래스까지 검색하지 않고 바로 \__dict__ 사전에서 값을 반환한다. 때문에 디스크립터 프로토콜이 사용되지 않고 다음에 이 속성을 조회할 때는 덮어써진 99 값을 반환한다.

그 뒤에 del을 호출해 이 속성을 지우면 객체의 \__dict__ 사전에서 descriptor 키를 지운 것과 같으므로 다시 앞의 시나리오로 돌아가게 된다. 즉 다시 디스크립터 프로토콜이 다시 활성화 된다.

In [94]:
del client.descriptor
vars(client)

{}

In [95]:
client.descriptor

42

이렇게 descriptor의 속성을 설정하면 우연히 속성이 깨지게 된다. 왜냐하면 디스크립터가 \__delete__ 메서드를 구현하지 않았기 때문이다.

이런 유형의 디스크립터는 다음 세션에서 볼 예정인 \__set__ 매직멧머드를 구현하지 않았기 때문에 비데이터 디스크립터라고 한다.

In [97]:
client.__dict__

{}

In [98]:
client.descriptor = 10

In [99]:
client.__dict__

{'descriptor': 10}

In [100]:
client.descriptor = client

In [101]:
client.__dict__

{'descriptor': <__main__.ClientClass at 0x6be7750>}

In [102]:
client.desc = 12

In [103]:
client.__dict__

{'descriptor': <__main__.ClientClass at 0x6be7750>, 'desc': 12}

In [104]:
del client.__dict__

In [105]:
client.__dict__

{}

In [106]:
client.descriptor

42

In [107]:
client.descriptor = client

In [108]:
client.descriptor

In [117]:
client.asss = 12

In [119]:
class TestClass:
    def __init__(self, data):
        self.data = data

In [120]:
testclass = TestClass('123')

In [121]:
testclass.data

'123'

In [122]:
testclass.good

AttributeError: 'TestClass' object has no attribute 'good'

In [123]:
testclass.good = 12

In [124]:
testclass.good

12

In [125]:
testclass.__dict__

{'data': '123', 'good': 12}

In [126]:
TestClass.asd = 123

In [127]:
vars(TestClass)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.TestClass.__init__(self, data)>,
              '__dict__': <attribute '__dict__' of 'TestClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'TestClass' objects>,
              '__doc__': None,
              'asd': 123})

In [128]:
test = TestClass('data')

In [129]:
test.__dict__

{'data': 'data'}

In [130]:
test.asd

123

In [131]:
del test.asd

AttributeError: asd

In [132]:
del TestClass.asd

In [133]:
test.__dict__

{'data': 'data'}

In [134]:
TestClass.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.TestClass.__init__(self, data)>,
              '__dict__': <attribute '__dict__' of 'TestClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'TestClass' objects>,
              '__doc__': None})

### 데이터 디스크립터

이제 데이터 디스크립터와의 차이를 살펴보자. 이를 위해 \__set__ 메서드를 구현한 또 다른 간단한 디스크립터를 생성할 것이다.

In [165]:
class DataDescriptor:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 42
    def __set__(self, instance, value):
        print("%s.descriptor를 %s 값으로 설정" %(instance, value))
        instance.__dict__["descriptor"] = value

In [166]:
class ClientClass:
    descriptor = DataDescriptor()

descriptor의 반환 값을 확인해보자.

In [167]:
client = ClientClass()

In [168]:
client.descriptor

42

In [169]:
client.descriptor = 99

<__main__.ClientClass object at 0x06C0B0B0>.descriptor를 99 값으로 설정


In [170]:
client.descriptor

42

descriptor의 반환 값이 변경되지 않았다. 그러나 다른 값으로 할당하면 앞의 예와 마찬가지로 객체의 \__dict__ 사전에는 업데이트 돼야 한다.

In [171]:
vars(client)

{'descriptor': 99}

In [172]:
client.__dict__["descriptor"]

99

이렇게 되는 이유는 사실 \__set__() 메서드가 호출되면 객체의 사전에 값을 설정하기 때문이다. 그리고 데이터 디스크립터에서 속성을 조회하면 객체의 \__dict__에서 조회하는 대신 클래스의 descriptor를 먼저 조회한다. 데이터 디스크립터는 인스턴스의 \__dict__를 오버라이드하여 인스턴스 사전보다 높은 우선순위를 가지지만, 비데이터 디스크립터는 인스턴스 사전보다 낮은 우선순위를 가진다.

속성 삭제는 더 이상 동작하지 않는다.

In [173]:
del client.descriptor

AttributeError: __delete__

삭제가 되지 않는 이유는 del을 호출하면 이제 인스턴스의 \__dict__에서 속성을 지우려고 시도하는 것이 아니라 descriptor에서 \__delete__() 메서드를 호출하게 되는데 이 예제에서는 \__delete__ 메서드를 구현하지 않았기 때문이다.

이것이 데이터 디스크립터와 비데이터 디스크립터의 차이이다. 만약 디스크립터가 \__set__() 메서드를 구현했다면 객체의 사전보다 높은 우선순위를 갖는다. \__set__() 메서드를 구현하지 않았다면 객체의 사전이 우선순위를 갖고 그 다음에 디스크립터가 실행된다. 

어쩌면 발견했을지도 모르는 흥미로운 코드가 있다.

In [175]:
instance.__dict__['descriptor'] = value

이 코드에는 살펴볼 내용이 많아 보인다. 하나씩 차례로 살펴보자. 

첫째 왜 하필 "desciptor"라는 이름의 속성 값을 바꾸는 것일까? 이 예제는 단순화를 위해 디스크립터의 이름을 따로 설정하지 않았기 때문이다. 

실제로는 \__init__메서드에서 디스크립터의 이름을 받아서 내부에 저장하거나 또는 \__set_name__ 메서드를 사용해 이름을 설정할 수 있다.

인스턴스의 \__dict__ 속성에 직접 접근하는 이유는 무엇일까? 적어도 두 가지의 설명이 있을 수 있다. 첫쨰로 왜 단순하게 다름처럼 하지 않았을까 하는 것이다.

In [178]:
setattr(instance, "descriptor", value)

디스크립터의 속성에 무언가 할당하려고 하면 \__set__ 메서드가 호출된다는 것을 기억하자. 따라서 setattr()을 사용하면 디스크립터의 \__set__ 메서드가 호출되고, \__set__메서드는 setattr을 호줄하고 다시 호출되는 무한루프가 발생한다.

instance.dscriptor = value와 같은 할당 표현식도 같은 이유로 무한루프를 유발한다.

그런데 디스크립터가 모든 인스턴스의 프로퍼티 값을 보관할 수 없는 이유는 뭘까?

클라이언트 클래스는 이미 디스크립터의 참조를 가지고 있다. 디스크립터가 다시 클라이언트 객체를 참조하면 순환 종속성이 생기게 되어 가비지 커렉션이 되지 않는 문제가 생긴다. 서로를 가리키고 있기 때문에 참조 카운트가 제거 임계치 이하로 떨어지지 않는다.

이에 대한 대안은 weakref 모듈에 있는 약한 참조를 사용하여 약한 참조 키 사전을 만드는 것이다. 약한 참조에 대한 내용은 잠시 후에 알아보자. 이러한 구현 방식을 따르는 것이 상당히 일반적이어서 이 책에서도 같은 방법을 사용한다.

## 디스크립터 실전

이제 디스크립터가 무엇인지 어떻게 작동하는지 그리고 그 뒤에 있는 주요 개념이 무엇인지 살펴보았다. 이제 디스크립터를 통해 처리할 수 있는 몇 가지 상황에 대해 살펴본다.

### 디스크립터를 사용한 애플리케이션

이번에 다룰 예제는 간단한 예로 시작하겠지만 결국은 코드 중복에 대한 얘기로 끝날 것이다. 이 무제를 어떻게 해결할지는 멸확하지 않다. 그러나 나중에 중복 코드를 디스크립터로 추상화하는 방법을 개발할 것이며 ㅋㄹ라이언트의 코드가 혁신적으로 줄어드는 것을 볼 수 있을 것이다.

### 디스크립터를 사용하지 않은 예

지금부터 살펴볼 예제는 속성을 가진 일반적인 클래스인데 속성의 값이 달라질 때마다 추적하려고 한다. 처음 떠오르는 해법은 속성의 setter 메서드에서 값이 변경될 때 검사하여 리스트와 같은 내부 변수에 값을 저장하는 것이다.

이번 애플리케이션에서 사용하는 클래스는 여행자를 표현하며 현재 어느 도시에 있는지를 속성으로 가진다. 프로그램을 실행하면서 사용자가 방문한 모든 도시를 추적할 것이다. 다음 코드와 같이 구현할  수 있다.

In [1]:
class Traveller:
    def __init__(self, name, current_city):
        self.name = name
        self._current_city = current_city
        self._cities_visited = [current_city]
    
    @property
    def current_city(self):
        return self._current_city
    
    @current_city.setter
    def current_city(self, new_city):
        if new_city != self._current_city:
            self._cities_visited.append(new_city)
            self._current_city = new_city
        
    @property
    def cities_visited(self):
        return self._cities_visited

In [4]:
alice = Traveller('Alice', 'Barcelona')
alice.current_city = 'Paris'
alice.cities_visited

['Barcelona', 'Paris']

이것이 필요한 전부라면 추가로 구현할 것은 없다. 프로퍼티를 사용하는 것만으로 충분하다. 그러나 애플리케이션의 여러 곳에서 똑같은 로직을 사용한다면 어떻게 될까? 속성의 모든 변수를 추적하는 것이 보다 일반적인 문제라 가정해보자. 

예를 들어 구입한 모든 티켓을 추적하거나 방문했던 모든 국가를 추적하는 등의 일을 하고 싶다면 모든 곳에서 같은 로직을 반복해야할 것이다. 게다가 다른 클래스에서도 같은 로직을 사용하려 한다면 어떻게 될까? 코드를 반복하거나 데코레이터, 프로퍼티 빌더 또는 디스크립터 같은 것을 만들어야 할 것이다. 프로퍼티 빌더는 디스크립터의 보다 복잡한 특별한 버전이다.

### 이상적인 구현방법

모든 클래스에 적용할 수 있도록 디스크립터를 사용하여 이전 섹션의 문제를 해결하는 방법을 살펴보자. 

이제 속성에 대해 이름을 가진 일반적인 디스크립터를 만들 것이다. 이 디스크립터는 값이 달라질 경우 리시트에 저장하여 추적하는 기능을 가진다.

이미 언급했듯이 만들 예제는 현재 필요한 것 이상의 기능을 제공하지만 디스크립터가 어떻게 유용하게 사용될 수 있는지 설명하기 위한 것이다. 디스크립터의 일반적인 특징이 그러하듯 메서드나 속성의 이름이 현재의 도메인 문제와 관련이 없음을 알 수 있다. 이는 디스크립터가 어떤 유형의 클래스 또는 다른 프로젝트에서도 동일한 결과를 내도록 설계되었기 때문이다.

이러한 이상적인 구현과 실전에서의 필요성에 대한 차이를 해결하기 위해 코드의 일부분에 주석 번호를 달고 각 코드의 기능과 원래 문제와의 관련성에 대한 설명이 추가되었다.

In [5]:
class HistoryTracedAttribute:
    def __init__(self, trace_attribute_name) -> None:
        self.trace_attribute_name = trace_attribute_name # [1]
        self._name = None
    
    def __sef_name__(self, owner, name):
        self._name = name
    
    def __get__(self, instance, owner):
        if instance is None: 
            return self
        return instance.__dict__[self._name]
    
    def __set__(self, instance, value):
        self._track_change_in_value_for_instance(instance, value)
        instance.__dict__[self._name] = value
    
    def _track_change_in_value_for_instance(self, instance, value):
        self._set_default(instance) # [2]
        if self._needs_to_track_change(instance, value):
            instance.__dict__[self.trace_attribute_name].append(value)
    
    def _needs_to_track_change(self, instance, value) -> bool:
        try:
            current_value = instance.__dict__[self._name]
        except KeyError: # [3]
            return True
        return value != current_value #[4]
    
    def _self_default(self, instance):
        instance.__dict__.setdefault(self.trace_attribute_name, []) # [6]
        
class Traveller:
    current_city = HistoryTracedAttribute("cities_visited") # [1]
    
    def __init__(self, name, current_city):
        self.name = name
        self.current_city = current_city # [5]

주석 번호에 대한 코드는 다음과 같다.

1. 속성의 이름은 디스크립터에 할당된 변수 중 하나로 여기서는 current_city이다. 그리고 이에 대한 추적을 저장할 변수의 이름을 디스크립터에 전달한다. 이 예에서는 cities_visited라는 속성에 current_city의 모든 값을 추적하도록 지시한다.

2. 디스크립터를 처음으로 호출할 때는 추적 값이 존재하지 않을 것이므로 나중에 추가할 수 있도록 비어있는 배열로 초기화한다. 

3. 처음 Travller를 호출할 때는 방문지가 없으므로 인스턴스 사전에서 current_city의 키도 존재하지 않을 것이다. 이런 경우도 새로운 여행지가 생긴 것이므로 추적의 대상이 된다. 앞에서 목록을 초기화하는 것과 비슷한 이유이다.

4. 새 값이 현재 설정된 값과 다른 경우에만 변경 사항을 추적한다.

5. Traveller의 \__init__ 메서드에서 디스크립터가 이미 생성된 단계이다. 할당 명령은 2단계 값을 추적하기 위한 빈 리스트 만들기를 실행하고, 3단계를 실행하여 리스트에 값을 추가하고 나중에 검색하기 위한 키를 설정한다 

6. setdefault는 KeyError를 피하기 우해 사용된다. setdefault는 두 개의 파라미터를 받는데 첫 번째 파라미터의 키가 있으면 해당 값을 반환하고 없으면 두 번쨰 파라미터를 반환한다.

디스크립터의 코드가 다소 복잡한 것은 사실이다. 반면에 클라이언트 클래스의 코드는 상당히 간단해졌다. 따라서 이 디스크립터를 여러 번 사용한다면 앞서 살펴본 것처럼 충분히 가치가 있을 것이다.

이 시점에서 궁금한 점은 디스크립터가 클라이언트 클래스와 완전히 독립적인가 하는 것이다. 디스클비터 안에서는 어떠한 비즈니스 로직도 포함되어 있지 않다. 따라서 완전히 다른 어떤 클래스에서 적용하여도 같은 효과를 낼 것이다. 

이것인 진정 파이썬스러운 디스크립터의 득징이다. 디스크립터는 비즈니스 로직의 구현보다는 라이브러리, 프레임워크 또는 내부 API를 정의하는데 적합하다.

### 다른 형태의 디스크립터

디스크립터의 구현 방법을 생각하기 전에 디스크립터의 특성과 관련된 문제를 먼저 이해해야 한다. 먼저 전역 상태 공유 문제에 대해 논의하고, 이후에 이 문제를 염두에 두고 어떻게 다양하게 구현할 수 있는지 살펴본다.

### 전역 상태 공유 변수

이미 살펴본 것처럼 디스크립터는 클래스 속성으로 설정해야 한다. 이것은 대배분의 경우에 큰 문제가 안되지만 몇 가지 고려해야 할 점이 있다. 클래스 속성의 문제점은 이들이 해당 클래스의 모든 인스턴스에서 공유된다는 것이다. 디스크립터도 예외가 아니기 때문에 디스크립터 객체에 데이터를 보관하면 모든 객체가 동일한 값에 접근할 수 있다.

각 객체에 데이터를 저장하는 대신 디스크립터가 데이터 자체를 유지하도록 잘못 정의하면 어떻게 되는지 살펴보자.

In [6]:
class SharedDataDescriptor:
    def __init__(self, initial_value):
        self.value = initial_value
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
    
class ClientClass:
    descriptor = SharedDataDescriptor("첫 번째 값")

이 예에서 디스크립터 객체는 데이터 자체를 바로 저장한다. 이것은 인스턴스의 값을 수정하면 같은 클래스의 다른 모든 인스턴스에서도 값이 수정된다는 것을 의미한다. 다음 코드는 그러한 예를 보여준다.

In [7]:
client1 = ClientClass()
client1.descriptor

'첫 번째 값'

In [8]:
client2 = ClientClass()
client2.descriptor

'첫 번째 값'

In [9]:
client2.descriptor = "client2를 위한 값"

In [11]:
client2.descriptor

'client2를 위한 값'

In [12]:
client1.descriptor

'client2를 위한 값'

한 객체의 값을 변경하면 갑자기 모든 객체의 값이 한꺼번에 변경되는 것을 볼 수 있다. 이것은 ClientClass.descriptor가 고유하기 때문이다. 이것은 모든 인스턴스에 대해 동일한 속성이다. 

어떤 경우에는 이것이 실제로 원하는 것일 수도 있다(예를 들어 클래스의 모든 객체에 대해 상태를 공유하려는 일종의 Borg 패턴을 구현한 경우). 그러나 객체를 구별하는 것이 일반적이다. 

이를 해결하기 위해서 디스크립터는 각 인스턴스의 값을 보관했다가 반환해야 한다. 이것이 각 인스턴스의 \__dict__ 사전에 값을 설정하고 검색하는 이유이다.

이렇게 해결하는 것이 가장 일반적인 방법이다. 이미 살펴본 것처럼  getattr()과 setattr()을 사용할 수 없기 때문에 \__dict__ 속성을 수정하는 것이 최후의 사용 가능한 선택이다. 

### 객체의 사전에 접근하기

디스크립터는 객체의 사전 \__dict__에 값을 저장하고 조회한다. 항상 인스턴스의 \__dict__ 속성에서 데이터를 저장하고 반환한다.

### 약한 참조 사용

\__dict__를 사용하지 않으려는 겨웅 또 다른 대안은 디스크립터 객체가 직접 내부 매핑을 통해 각 인스턴스의 값을 보관하고 반환하는 것이다.

이렇게 하는 것에는 주의사항이 있다. 내부 매핑을 할 때 사전을 사용하면 안 된다. 클라이언트 클래스는 디스크립터에 대한 참조를 가지며 디스크립터는 디스크립터를 사용하는 객체에 대한 참조를 가지므로 순환 종속성이 생겨 결과적으로 결코 가비지 컬렌션이 되지 않는 문제가 있다. 

이를 해결하기 위해 사전은 weakref(WEAKREF 01) 모듈에 정의된 것처럼 약한 키가 되어야 한다.

이 경우 디스크립터의 코드는 다음과 같다.

In [13]:
from weakref import WeakKeyDictionary

class DescriptorClass:
    def __init__(self, initial_value):
        self.value = initial_value
        self.mapping = WeakKeyDictionary()
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.mapping.get(instance, self.value)
    
    def __set__(self, instance, value):
        self.mapping[instance] = value

이렇게 하면 문제가 해결되지만 몇 가지 고려사항이 있다.

* 인스턴스 객체는 더 이상 속성을 보유하지 않는다. 대신 디스크립터가 속성을 보유한다. 이것은 다소 논란의 여지가 있으며 개념적 관점에서 보면 완전히 정확하지 않을 수도 있다. 이 세부 사항을 잊어버리면 객체의 사전에 있는 내용을 찾으려고 할 수 있으나(vars) 객체는 속성을 보유하지 않았기 때문에 완전한 데이터를 반환하지 않을 것이다.

* 객체는 \__hash__ 메서드를 구현하여 해시가 가능해야 한다. 만약 해시가 가능하지 않다면 WeakKeyDictionary에 매핑할 수가 없다. 어떤 애플리케이션에서는 이것이 너무 엄격한 요구 사항일 수도 있다.

### 디스크립터에 대한 추가 고려사항

디스크립터를 사용하는 것이 좋은 선택일 경우 어떤 일을 할 수 있는지 그리고 처음에 접근했던 방법을 디스크립터를 사용해 어떻게 개선할 수 있는지에 대한 관점에서 일반적인 고려사항을 검토한다. 디스크립터를 사용해 구현한 다음에는 원래의 구현과 비교해 장단점을 비교해본다. 

### 코드 재사용

디스크립터는 코드 중복을 피하기 위한 일반적인 도구이자 강력한 추상화 도구이다. 디스크립터가 필요한 곳을 찾는 가장 좋은 방법은 프로퍼티가 필요한 구조가 반복되는 경우를 찾는 것이다.

프로퍼티는 단지 디스크립터의 특수한 경우일 뿐이다(@property 데코레이터는 get, set 및 delete 작업을 정의하여 디스크립터 프로토콜을 모두 구현한 디스크립터이다). 즉 디스크립터는 프로퍼티보다 훨씬 복잡한 작업에 사용도리 수 있음을 뜻한다.

디스크립터는 데코레이터가 클래스 메서드에서도 동작할 수 있도록 도와 더 나은 데코레이터를 만들 수 있게 한다.

데코레이터는 항상 \__get__() 메서드를 구현하고 디스크립터를 사용하는 것이 안전하다고 말할 수 있다.

일반적인 디스크립터에 대해서는 3의 규칙 외에도 기억할 것이 있다. 클라이언트가 사용하게 되는 내부 API에 대해서는 디스크립터를 사용하는 것이 좋다는 것이다. 이는 일회성 솔루션이 아닌 라이브러리나 프레임워크의 디자인에 대해서는 기능을 확장하기 좋기 때문이다.

디스크립터에 비즈니스 로직을 넣으면 안 된다. 디스크립터의 코드에는 비즈니스 코드가 아닌 구현 코드가 더 많이 포함되어야 한다. 디스크립터는 비즈니스 로직에서 사용할 새로운 객체나 데이터 구조를 정의하는 것과 비슷하다.

### 클래스 데코레이터 피하기

두 개의 클래스 데코레이터를 사용하여 구현한 것을 디스크립터로 변경할 수 있을까?

In [15]:
@Serialization(
    username=show_original,
    password=hide_field,
    ip=show_original,
    timestamp=format_time,
)

@dataclass
class LoginEvent:
    username: str
    password: str
    ip: str
    timestamp: datetime

주요 내용은 각 속성의 값에 대해 요건에 맞게 변환 후에 수정된 버전을 반환하는 디스크립터를 만드는 것이다. 예를 들어 날짜를 정확하게 포매팅하는 등의 작업이나 민감한 정보를 숨기는 것이다.

In [16]:
from functools import partial
from typing import Callable

class BaseFieldTransformation:
    def __init__(self, transformation:Callable[[], str]) -> None:
        self._name = None
        self.transformation = transformation
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        raw_value = instance.__dict__[self._name]
        return self.transformation(raw_value)
    
    def __set_name__(self, owner, name):
        self._name = name
    
    def __set__(self, instance, value):
        instance.__dict__[self._name] = value
    
ShowOriginal = partial(BaseFieldTransformation, transformation=lambda x: x)
HideField = partial(BaseFieldTransformation, transformation=lambda x: "**민감한 정보 삭제**")
FormatTime = partial(
    BaseFieldTransformation,
    transformation=lambda ft:ft.strftime("%Y-%m-%d %H:%M")
)

하나의 파라미터를 취하여 하나의 값을 반환하는 함수를 가진 흥미로운 디스크립터이다. 이 함수는 필드에 적용하려는 변환 함수이다. 디스크립터 클래스의 정의와 함께 필요한 규칙이 정의되어 있는 디스크립터 클래스가 있다.

이 예에서는 하위 클래스를 추가 생성하는 방법으로 functools.partial를 사용하고 있다. 클래스 변환 함수에 호출 가능한 함수를 직접 전달하여 함수의 새 버전을 만들었다.

예제를 단순하게 하기 위해 \__init__()과 serialize() 메서드 또한 추상화할 수 있지만 직접 구현할 것이다. 이러한 배경 아래 이벤트 클래스는 다음과 같이 정의된다.

In [17]:
class LoginEvent:
    username = ShowOriginal()
    password = HideField()
    ip = ShowOriginal()
    timestamp = FormatTime()
    
    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp
    
    def serialize(self):
        return { 
            "username":self.username, 
            "password":self.password, 
            "ip":self.ip, 
            "timestamp":self.timestamp
        }

위 코드를 실행하면 다음과 같이 출력된다.

In [23]:
from datetime import datetime

le = LoginEvent('john', 'secret password', '1.1.1.1', datetime.utcnow())

In [25]:
le.serialize()

{'username': datetime.datetime(2020, 9, 21, 0, 58, 29, 802048),
 'password': '**민감한 정보 삭제**',
 'ip': datetime.datetime(2020, 9, 21, 0, 58, 29, 802048),
 'timestamp': '2020-09-21 00:58'}

In [28]:
le.password

'**민감한 정보 삭제**'

데코레이터를 사용한 이전 구현과 비교하면 몇 가지 차이점이 있다. 이 예제에서는 serialize() 메서드를 추가하고 필드를 결과 사전에 표시하기 전에 숨겼다. 그러나 이 순간에도 메모리의 이벤트 인스턴스에서 변환을 적용하지 않은 원래의 값을 구할 수 있다. 물론 값을 설정할 때 미리 변환한 값을 저장하고 가져올 때는 그대로 가져올 수도 있다.

애플리케이션의 민감도에 따라 허용되거나 허용되지 않을 수도 있지만, 이번 예제에서는 객체의 public 속성을 요청하면 디스크립터가 결과를 보여주기 전에 변환 작업을 적용한다. 객체의 \__dict__ 사전에 접근하여 원본 값을 가져올 수도 있다. 그러나 기본적으로 값을 요청하면 변환된 값을 반환한다.

이 예제에서 모든 디스크립터는 기본 클래스에 정의된 공통 논리를 따른다. 디스크립터는 객체에 값을 저장한 다음 정의된 변환 로직에 따라 값을 반환한다. 템플릿 메서드 디자인 패턴으로 클래스마다 고유한 변환 함수를 갖도록 클래스 계층 구조를 만들 수도 있다. 그러나 이번 예제의 경우 파생 클래스의 변경 사항이 상대적으로 적기 때문에 파생 클래스를 기본 클래스의 부분 애플리케이션으로 만드는 방법을 선택했다. 새로운 변환 필드를 생성하는 것은 새로운 기본 클래스를 정의하는 것만큼 간단해야 한다. 기존 클래스에 기반을 둔 새로운 클래스는 간단하게 정의할 수 있으므로 이름을 설정할 필요가 없다.

이 구현과 관계없이 디스크립터는 객체이므로 모델을 만들어서 객체 지향 프로그래밍의 모든 규칙을 적용할 수 있다. 디자인 패턴은 디스크립터에도 적용된다. 계층 구조를 정의하고 사용자 정의 동작을 설정하는 등의 작업을 할 수 있다. 이 예제는 OCP를 따른다. 새로운 변환 기능을 추가할 때 기본 클래스를 수정하지 않고 파생 클래스를 만들면 되기 때문이다. 앞의 데코레이터 구현도 OCP를 따르지만 각 변환 메커니즘에 클래스가 사용되지는 않았다.

\__init__()과 serialize() 메서드를 구현한 기본 클래스를 만들고, 다음과 같이 그것을 상속받아 LoginEvent 클래스를 간단히 정의하는 방법을 살펴보자.

In [31]:
class BaseEvent:
    def __init__(self):
        pass
    def serialize(self):
        pass

class LoginEvent(BaseEvent):
    username = ShowOriginal()
    password = HideField()
    ip = ShowOriginal()
    timestamp = FormatTime()

이렇게 코드를 작성하면 클래스가 훨씬 깔끔해 보인다. 필요한 속성만 정의하면 되고 각 속성의 클래스를 보면 어떤 로직이 적용되었는지 바로 이해할 수 있다. 기본 클래스는 공통 메서드만 추상화할 것이고, 결과적으로 각 이벤트 클래스는 더 작고 간단하게 된다.

각 이벤트 클래스가 단순해질 뿐 아니라, 디스크립터 자체도 매우 작아서 클래스 데코레이터보다 훨씬 간단하다. 클래스 데코레이터를 사용한 원래의 방식도 좋았지만 디스크립터를 사용한 방식이 보다 더 뛰어나다.

## 디스크립터 분석

디스크립터를 사용하여 구현이 깨끗해지고 정확해졌다는 것을 어떻게 알 수 있을까? 좋은 디스크립터의 기준은 무엇일까? 우리는 이 도구를 적절히 사용하는 것일까? 아니면 오버 엔지니어링을 하는 것일까?

이러한 질문에 대답하기 위해 디스크립터를 보다 자세히 분석해볼 것이다.

### 파이썬 내부에서의 디스크립터 활용

어떤 것이 좋은 디스크립터인지 확인하는 가장 간단한 방법은 다른 훌륭한 파이썬 객체와 얼마나 유사한지를 보는 것이다. 좋은 디스크립터는 파이썬 자체의 디스크립터와 동일하다. 따라서 파이썬이 어떻게 디스크립터를 사용하는지 분석해보면 훌륭한 구현을 통해 기대할 수 있는 점을 알 수 있게 된다.

파이썬이 내부 로직의 일부를 해결하기 위해 디스크립터를 사용하는 가장 일반적인 시나리오를 살펴볼 것이다. 이 과정에서 우아하고도 평범하게 존재하는 디스크립터를 발견하게 될 것이다.

### 함수와 메서드

디스크립터 객체 중에 가장 멋있는 예를 바로 함수일 것이다. 함수는 \__get__ 메서드를 구현했기 때문에 클래스 안에서 메서드처럼 동작할 수 있다. 

메서드는 추가 파라미터를 가진 함수일 뿐이다. 관습적으로 메서드의 첫 번재 파라미터는 "self"라는 이름을 사용하며 메서드를 소유하고 있는 클래스의 인스턴스를 나타낸다. 따라서 메서드에서 "self"를 사용하는 것은 객체를 받아서 수정을 하는 함수를 사용하는 것과 동일하다. 

다른 말로하면 다음과 같이 정의한 경우

In [38]:
class MyClass:
    def method(self):
        self.x = x

실제로 다음과 같이 정의하는 것과 같다.

In [39]:
class MyClass: pass

def method(myclass_instance):
    myclass_instance.x =1
    
method(MyClass())

따라서 메서드는 객체를 수정하는 또 다른 함수일 뿐이며, 객체 안에서 정의되었기 때문에 객체에 바인딩되어 있다고 말한다.

다음과 같은 형태로 호출하면

In [41]:
instance = MyClass()
instance.method()

파이썬은 실제로 다음과 같이 처리한다.

In [43]:
instance = MyClass()
MyClass.method(instance)

이것은 파이썬에 의해 디스크립터의 도움을 받아 내부적으로 처리되는 구문 변환일 뿐이라는 것에 유의하자.

함수는 디스크립터 ㅍ로토콜을 구현했으므로 메서드를 호출하기 전에 \__get__() 메서드가 먼저 호출되고 필요한 변환을 한다. 

In [44]:
def function():pass

function.__get__

<method-wrapper '__get__' of function object at 0x05580738>

instance.method() 구문에서는 괄호 안의 인자를 처리하기 전에 "instance.method" 부분이 먼저 평가된다. 

method는 클래스 속성으로 정의된 객체이고 \__get__ 메서드가 있기 때문에 \__get__ 메서드가 호출된다. 그리고 \__get__ 메서드가 하는 일은 함수를 메서드로 변환하는 것이다. 즉 함수를 작업하려는 객체의 인스턴스에 바인딩한다.

파이썬이 내부적으로 하는 일에 대해 좀 더 자세히 알아볼 수 있도록 예제를 살펴보자.

외부에서 호출 가능한 형태의 함수 또는 메서드를 클래스 내에 호출 가능한 객체로 정의할 것이다. Method 클래스의 인스턴스는 함수나 메서드 형태로 다른 클래스에서 사용될 것이다. 이 함수는 단지 전달 받은 3개의 인자를 그대로 출력한다. 첫 번째 파라미터는 instance로 클래스에 정의된 겨웅 self가 된다. 그리고 \__call__() 메서드에서 self는 MyClass의 인스턴스가 아니라 Method의 인스턴스를 나타내는 것에 주의하자. 파라미터로 전달된 instance가 MyClass 탕비의 객체이다.

In [46]:
class Method:
    def __init__(self, name):
        self.name = name
    
    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} 호출됨. 인자는 {arg1}와 {arg2}입니다.")

class MyClass:
    method = Method("Internal call")

In [47]:
instance = MyClass()

In [48]:
Method("External call")(instance, "1", "2")

External call: <__main__.MyClass object at 0x05816910> 호출됨. 인자는 1와 2입니다.


In [50]:
instance.method("1", "2")

TypeError: __call__() missing 1 required positional argument: 'arg2'

오류가 발생한 이유는 파라미터의 위치가 한 칸씩 밀려서 Method.\__call__ 기준으로 self 자리에 instance가 전달되고, instance 자리에 "first"가 전달되고, arg1 자리에 "second"가 전달되는데, arg2 자리에는 아무 값도 전달되지 않았기 때문이다.

이 문제를 해결하려면 메서드르 디스크립터로 변경하면 된다.

그렇게 하면 instance.method 호출 시 Method.\__get__ 메서드를 먼저 호출할 것이다. 여기에서 첫 번째 파라미터로 Method의 인스턴스를 전달함으로써 객체에 바인딩하면 된다.

In [56]:
from types import MethodType

class Method:
    def __init__(self, name):
        self.name = name
    
    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} 호출됨. 인자는 {arg1}, {arg2}입니다.")
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return MethodType(self, instance)

class MyClass:
    method = Method("Internal call")

In [57]:
Method("External call")(instance, "1", "2")

External call: <__main__.MyClass object at 0x0586A990> 호출됨. 인자는 1, 2입니다.


In [58]:
instance = MyClass()
instance.method("1", "2")

Internal call: <__main__.MyClass object at 0x05959ED0> 호출됨. 인자는 1, 2입니다.


수정한 것은 types 모듈의 MethodType을 사용하여 함수를 메서드로 변환하는 것이다. 이 클래스의 첫 번째 파라미터는 호출 가능한 것이어야 한다(여기에서는 self인데 self는 Method의 인스턴스로 \__call__ 메서드를 구현했으므로 호출 가능한 형태이다). 두 번째 파라미터는 이 함수에 바인딩할 객체이다.

파이썬의 함수 객체도 이것과 비슷하게 동작한다. 따라서 클래스 내부에 함수를 정의할 경우 메서드처럼 사용할 수 있는 것이다. 

사용자 정의 객체를 만들 때도 이러한 파이썬스러운 접근 방식을 염두에 두는 것이 좋다. 예를 들어 사용자 정의 호출 가능한 객체를 정의할 때는 지금처럼 디스크립터로 만들어서 클래스 속성으로도 사용할 수 있도록 하는 것이 좋다.

### 메서드를 위한 빌트인 데코레이터

공식 문서(PYDESCR-02)에 설명된 것처럼 @property, @classmethod와 @staticmethod 데코레이터는 디스크립터이다.

메서드를 인스턴스가 아닌 클래스에서 직접 호출할 때는 관습적으로 디스크립터 자체를 반환한다는 것을 몇 차례 언급했었다. 프로퍼티를 클래스에서 직접 호출하면 계산할 속성이 없으르모 일종의 디스크립터인 프로퍼티 객체 자체를 반환한다.

@classmethod를 사용하면 디스크립터의 \__get__ 함수가 메서드를 인스턴스에서 호출하든 클래스에서 직접 호출하든 상관없이 데코레이팅 함수에 첫 번째 파라미터로 메서드를 소유한 클래스를 넘겨준다. 

@staticmethod를 사용하면 정의한 파라미터 이오의 파라미터를 넘기지 않도록 한다. 즉, \__get__ 메서드에서 함수의 첫 번째 파라미터에 self를 바인딩하는 작업을 취소한다.

### 슬롯(slot)

클래스에 \__slot__ 속성을 정의하면 클래스가 기대하는 특정 속성만 정의하고 다른 것은 제한할 수 있다. 

\__slot__에 정의되지 않은 속성을 동적으로 추가하려고 할 경우 AttributeError가 발생한다. 이 속성을 정의하면 클래스는 정적으로 되고 \__dict__ 속성을 갖지 않는다. 따라서 객체에 동적으로 속성을 추가할 수 없다.

그렇다면 객체의 사전이 없는데 어떻게 속성을 가져올 수 있을까? 바로 디스크립터를 사용하는 것이다. \__slof__에 정의된 이름마다 디스크립터를 만들어서 값을 저장하고 있으므로 나중에 검색도 가능하다.

In [60]:
class Coordinate2D:
    __slots__ = ('lat', 'long')
    
    def __init__(self, lat, long):
        self.lat = lat
        self.long = long
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self.lat}, {self.long})"

이것은 흥미로운 기능이지만 파이썬의 동적인 특성을 없애기 때문에 조심해서 사용해야 한다. 일반적으로 이것은 정적인 객체에만 사용해야 하며 코드의 다른 부분에서 절대로 동적으로 속성을 추가할 일이 없다는 것을 확신할 때에만 사용해야 한다. 

슬롯을 사용한 객체는 메모리는 덜 사용한다는 점이 장점이다. 